# Generate figures for SW WA wind biases

In [37]:
import numpy as np
import xarray as xr

from scores.continuous import mse
from scores.processing import broadcast_and_match_nan
from scores.stats.statistical_tests import diebold_mariano

from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [38]:
official = xr.open_dataarray("../data/sw_wa_wind/Official_WindMag_00_20230601-20230830_sw_wa.nc")
# Note that the 12Z AutoFcst was the automated guidance available to meteorologists for the
# afternoon (00Z) official forecast issue.
autofcst = xr.open_dataarray("../data/sw_wa_wind/AutoFcst_WindMag_12_20230601-20230830_sw_wa.nc")
obs = xr.open_dataarray("../data/sw_wa_wind/obs_WindMag_20230601-20230830sw_wa.nc")

# Match missing data between datasets
official, autofcst, obs = broadcast_and_match_nan(official, autofcst, obs)

### Calculate MSE and multiplicative bias

In [39]:
official_mse = mse(official, obs, preserve_dims="lead_day")
autofcst_mse = mse(autofcst, obs, preserve_dims="lead_day")

official_bias = official.mean(["valid_start", "station_number"]) / obs.mean(["valid_start", "station_number"])
autofcst_bias = autofcst.mean(["valid_start", "station_number"]) / obs.mean(["valid_start", "station_number"])

In [40]:
official_line_colour = 'rgba(230,159,0,1)'
autofcst_line_colour = 'rgba(86,180,233,1)'
figure = make_subplots(rows=2, cols=1, subplot_titles=("<b>(a)</b>", "<b>(b)</b>"), vertical_spacing = 0.15)
figure.update_annotations(font_size=12, xshift=-160, xanchor="left")

# Upper subfig - MSE
figure.add_trace(
    go.Scatter(
        x=official_mse.lead_day,
        y=official_mse.values,
        line=dict(color=official_line_colour),
        name="Official"
    ), row=1, col=1
)
figure.add_trace(
    go.Scatter(
        x=autofcst_mse.lead_day,
        y=autofcst_mse.values,
        line=dict(color=autofcst_line_colour),
        name="Automated"
    ), row=1, col=1
)

# Lower subfig - multiplicative bias
figure.add_trace(
    go.Scatter(
        x=[1, 7],
        y=[1, 1],
        line=dict(color="black", dash='dash'),
        mode='lines',
        name="Automated",
        showlegend=False
    ), row=2, col=1
)
figure.add_trace(
    go.Scatter(
        x=official_bias.lead_day,
        y=official_bias.values,
        line=dict(color=official_line_colour),
        name="Official",
        showlegend=False
    ), row=2, col=1
)
figure.add_trace(
    go.Scatter(
        x=autofcst_bias.lead_day,
        y=autofcst_bias.values,
        line=dict(color=autofcst_line_colour),
        name="Automated",
        showlegend=False
    ), row=2, col=1
)

figure.update_layout(
    legend=dict(x=0.01,y=0.99),
    height=600,
    width=400,
    margin=go.layout.Margin(
        l=20, #left margin
        r=20, #right margin
        b=20, #bottom margin
        t=20, #top margin
    ),
)
figure.update_xaxes(title_text="Lead day", row=1, col=1, tickmode="linear", tick0=0, dtick=1)
figure.update_xaxes(title_text="Lead day", row=2, col=1, tickmode="linear", tick0=0, dtick=1)
figure.update_yaxes(title_text="MSE (kt<sup>2</sup>)", row=1, col=1)
figure.update_yaxes(title_text="Multiplicative bias", row=2, col=1)

In [41]:
figure.write_image("../figures/sw_wa_wind_bias.svg")

### Check statistical significance

In [42]:
official_mse = mse(official, obs, preserve_dims=["lead_day", "valid_start"])
autofcst_mse = mse(autofcst, obs, preserve_dims=["lead_day", "valid_start"])

In [43]:
# Difference between Official and the hindcast
diff= official_mse - autofcst_mse
diff = diff.assign_coords(h=("lead_day", [2, 3, 4, 5, 6, 7, 8]))
dm_result = diebold_mariano(diff, "lead_day", "h")
dm_result

<xarray.Dataset>
Dimensions:          (lead_day: 7)
Coordinates:
  * lead_day         (lead_day) int64 1 2 3 4 5 6 7
Data variables:
    mean             (lead_day) float64 -1.927 -1.985 -1.888 ... -1.054 -0.1216
    dm_test_stat     (lead_day) float64 -4.063 -4.615 -3.38 ... -0.8742 -0.1069
    timeseries_len   (lead_day) int64 2184 2184 2184 2184 2184 2184 2184
    confidence_gt_0  (lead_day) float64 2.42e-05 1.961e-06 ... 0.191 0.4574
    ci_upper         (lead_day) float64 -0.9975 -1.142 -0.7933 ... 1.309 2.109
    ci_lower         (lead_day) float64 -2.857 -2.828 -2.984 ... -3.416 -2.352